# KDDCup99 10% データによる異常検知評価
- KDDCup99 10% データをネットワークからダウンロードし、異常検知の評価を実施します。
- なおこのサンプルの実行には python(3.6), tensorflow, pandas, numpy, sklearn が必要です。

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

from dagmm import DAGMM

## データインポート

In [2]:
url_base = "http://kdd.ics.uci.edu/databases/kddcup99"

# KDDCup 10% Data
url_data = f"{url_base}/kddcup.data_10_percent.gz"
# データ属性情報（列名・列の型）
url_info = f"{url_base}/kddcup.names"

In [3]:
# データ属性情報を読み込み
df_info = pd.read_csv(url_info, sep=":", skiprows=1, index_col=False, names=["colname", "type"])
colnames = df_info.colname.values
coltypes = np.where(df_info["type"].str.contains("continuous"), "float", "str")
colnames = np.append(colnames, ["status"])
coltypes = np.append(coltypes, ["str"])

# データ本体のインポート
df = pd.read_csv(url_data, names=colnames, index_col=False,
                 dtype=dict(zip(colnames, coltypes)))

In [4]:
# データダミー化
X = pd.get_dummies(df.iloc[:,:-1]).values

# 目的変数の生成
# "normal" の場合、異常(1)とする。そうでない場合に 正常(0)
# 通常の考え方と逆だが、論文の趣旨に準じる
y = np.where(df.status == "normal.", 1, 0)

In [5]:
# 学習/評価用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=123)
X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]

## DAGMM データ学習
但し次の点を論文と変更しました:
- $\lambda_2$ を 0.0001 としました (論文では 0.005)
- GMM の共分散行列の対角成分に小さな値 ($10^{-6}$) を加えて、特異にならないようにした（論文では特に言及無し）

これはデータの分布、および学習前の正規化(平均0, 分散1となるようにするなど)の方法に依存するものと思われます。  
（論文と同じパラメータでは、あまり良い精度とならなかった）

In [6]:
model = DAGMM(
    comp_hiddens=[60, 30, 10, 1], comp_activation=tf.nn.tanh,
    est_hiddens=[10, 4], est_dropout_ratio=0.5, est_activation=tf.nn.tanh,
    learning_rate=0.0001, epoch_size=200, minibatch_size=1024, random_seed=1111
)

In [7]:
model.fit(X_train)

 epoch 100/200 : loss = 80.526
 epoch 200/200 : loss = 72.563


## 学習済みモデルを検証データに適用

In [17]:
y_pred = model.predict(X_test)

In [18]:
# エネルギーのしきい値は、全データのエネルギー分布の80%点（上側20%点）
anomaly_energy_threshold = np.percentile(y_pred, 80)
print(f"Energy thleshold to detect anomaly : {anomaly_energy_threshold:.3f}")

Energy thleshold to detect anomaly : 6.518


In [19]:
# 検証データの異常判定
y_pred_flag = np.where(y_pred >= anomaly_energy_threshold, 1, 0)

In [20]:
prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
print(f" Precision = {prec:.3f}")
print(f" Recall    = {recall:.3f}")
print(f" F1-Score  = {fscore:.3f}")

 Precision = 0.932
 Recall    = 0.942
 F1-Score  = 0.937
